In [9]:
import sys
sys.path.append('/dccstor/hoo-misha-1/wilds/WOODS')

import numpy as np
import torch
import os

from scripts.utils import *

In [10]:
# for model in os.listdir('/dccstor/hoo-misha-1/wilds/wilds/pretrained/iwildcam'):
#     model = model[9:-4]
#     save_path = f'/dccstor/hoo-misha-1/wilds/wilds/pretrained/iwildcam/iwildcam_{model}.pth'
#     d = torch.load(save_path, map_location=torch.device('cpu'))
#     alg = d['algorithm']
#     if 'model.classifier.weight' in alg:
#         weight = alg['model.classifier.weight']
#         bias = alg['model.classifier.bias']
#     elif 'classifier.weight' in alg:
#         weight = alg['classifier.weight']
#         bias = alg['classifier.bias']
#     elif 'model.fc.bias' in alg:
#         weight = alg['model.fc.weight']
#         bias = alg['model.fc.bias']
#     elif 'model.1.weight' in alg:
#         weight = alg['model.1.weight']
#         bias = alg['model.1.bias']
#     else:
#         raise Exception(f"Cannot find classifier for {model}")
#     np.save(f'/dccstor/hoo-misha-1/wilds/WOODS/classifiers/iwildcam/iwildcam_{model}_classifier_weight.npy', weight)
#     np.save(f'/dccstor/hoo-misha-1/wilds/WOODS/classifiers/iwildcam/iwildcam_{model}_classifier_bias.npy', bias)


In [11]:
set_path_base(f'/dccstor/hoo-misha-1/wilds/wilds/features/iwildcam/ERM')

In [12]:
rng = np.random.default_rng(2022)
random_state = 0

In [13]:
set_random_state(rng, random_state)

In [14]:
f,l,m = load_flm()

In [58]:
f,l,m = cam_flm(1, [58])
f,l,m = prune_flm(f,l,m, 25)

In [20]:
weighted_iterative_kmeans_sample_ind(f,l,5)


UnboundLocalError: local variable 'num_shots_classes_center_labels' referenced before assignment

In [23]:
import time

In [59]:
def func(features, labels, num_shots=5, n=100):
    #start_time = time.perf_counter()
    unique_classes = np.unique(labels)
    num_classes = len(unique_classes)
    #print(features.shape, n*num_classes)
    num_shots_classes_kmeans = KMeans(n_clusters=n*num_classes, random_state=random_state).fit(features)
    #print(f'First kmeans: {time.perf_counter() - start_time}')
    start_time = time.perf_counter()
    num_shots_classes_centers = num_shots_classes_kmeans.cluster_centers_
    kmeans = KMeans(n_clusters=num_classes, random_state=random_state).fit(num_shots_classes_centers)
    #print(f'Second kmeans: {time.perf_counter() - start_time}')
    #start_time = time.perf_counter()
    centers = copy.copy(kmeans.cluster_centers_)
    #print(f'Copy kmeans centers: {time.perf_counter() - start_time}')
    #start_time = time.perf_counter()
    num_shots_classes_center_labels = [-1]*len(num_shots_classes_centers)
    center_labels = [-1] * len(centers)
    
    for i in range(len(features)):
        feature = features[i]
        try:
            ii = centers.index(feature)
            center_labels[ii] = labels[i]
            
        except:
            pass
        
        try:
            ii = num_shots_classes_centers.index(feature)
            num_shots_classes_center_labels[ii] = labels[i]
        except:
            pass
    #print(f'Generate labels: {time.perf_counter() - start_time}')
    #start_time = time.perf_counter()
    mean_dist = {}
    
    def get_dist(mu_m, mu_is, mu_is_labels):
        sum_d = 0
        
        mu_is_labels_counts = np.unique(mu_is_labels, return_counts=True)
        
        mu_is_labels_count_dict = {}
        for label_i in range(len(mu_is_labels_counts[0])):
            label = mu_is_labels_counts[0][label_i]
            counts = mu_is_labels_counts[1][label_i]
            mu_is_labels_count_dict[label] = counts
        
        for mu_i_i in range(len(mu_is)):
            mu_i = mu_is[mu_i_i]
            mu_i_label = mu_is_labels[mu_i_i]
            
            mu_i_label_count = mu_is_labels_count_dict[mu_i_label]
            if 1 - mu_i_label_count / len(mu_is) < 0:
                print(mu_i_label-count, len(mu_is))
            sum_d += (1 - mu_i_label_count / len(mu_is)) * np.linalg.norm(mu_m - mu_i)
        return sum_d

    shots_count = 0
    while(shots_count < num_shots*num_classes):
        max_dist = -1
        max_dist_i = None
        
        #center_labels_counts = np.unique(center_labels)
        
        for mu_m_i in range(len(num_shots_classes_centers)):
            
            mu_m = num_shots_classes_centers[mu_m_i]
            #mu_m_label = num_shots_classes_center_labels[mu_m_i]

            #mu_m_label_count_i = center_labels_counts[0].index(mu_m_label)
            #mu_m_label_count = center_labels_counts[1][mu_m_label_count_i]
            
            if mu_m in centers:
                continue
            #mu_m_dist = mu_m_label_count / len(center_labels) * get_dist(mu_m, centers)
            mu_m_dist = get_dist(mu_m, centers, center_labels)
            if mu_m_dist > max_dist:
                max_dist = mu_m_dist
                max_dist_i = mu_m_i
        if max_dist_i is None:
            print(max_dist, mu_m_dist)
        try:
            centers = np.vstack((centers, num_shots_classes_centers[max_dist_i][np.newaxis,:]))
        except:
            print(num_shots_classes_centers[max_dist_i].shape, num_shots_classes_centers[max_dist_i][np.newaxis,:].shape)
            centers = np.vstack((centers, np.squeeze(num_shots_classes_centers[max_dist_i])))
        center_labels.append(num_shots_classes_center_labels[max_dist_i])
        #ret_ind.append(max_dist_i)
        shots_count += 1
    #print(f'Get shots {time.perf_counter() - start_time}')
    #start_time = time.perf_counter()
    ret_ind = []  
    for center_index in range(centers.shape[0]):
        center = centers[center_index]
        min_dist = 9999
        min_i = None
        for feature_index in range(features.shape[0]):
            feature = features[feature_index]
            dist = np.linalg.norm(feature-center)
            if dist < min_dist:
                min_dist = dist
                min_i = feature_index
        ret_ind.append(min_i)
    #print(f'Finish {time.perf_counter() - start_time}')
    return np.array(ret_ind)

In [61]:
func(f,l,13)

array([103, 442, 273, 443,  51, 103, 480, 472, 341,  43, 202, 408, 386,
       379,  45, 382, 141, 179, 477,  23, 207,  13, 346, 419, 512, 184,
       258,  68, 152, 487, 154, 416, 288, 397,   7, 236,  80,  25, 394,
       423,  31, 117, 426, 395, 454, 174, 320, 264])

In [62]:
f.shape

(515, 2048)

In [63]:
len(np.unique(l))

3